In [1]:
# standard libraries
import numpy as np
from pprint import pprint as pp
from matplotlib.pyplot import *
%matplotlib inline

In [2]:
# user library
from bayesee.detector.spotter import Spotter
from bayesee.imaging.filter import *
from bayesee.imaging.image import *
from bayesee.operation.nb2d import *
from bayesee.operation.dp import *

In [3]:
# stimulus parameters
row = col = 128
ppd = 120
cpd = 1.5
cpi_v = row * cpd / ppd
field_ratio = 8

In [4]:
# simulation parameters
n_background = 1000
n_iter = 10
dp_tar = 1

In [5]:
# targets
rc_target = hann_window(row, col, row/2)
rc_target /= nb2dot(rc_target, rc_target)

sine_target = hann_window(row, col, row/2) * sine_wave(row, col, (cpi_v,0))
sine_target /= nb2dot(sine_target, sine_target)

tri_target = hann_window(row, col, row/2) * sine_triangle_wave(row, col, (cpi_v, 0))
tri_target /= nb2dot(tri_target, tri_target)

sqr_target = hann_window(row, col, row/2) * sine_square_wave(row, col, (cpi_v, 0))
sqr_target /= nb2dot(sqr_target, sqr_target)

In [6]:
# backgrounds
uni_images, field = power_noise_patches(row, col, row*field_ratio, col*field_ratio, -1, n_background)
pre_cm_images, _ = power_noise_patches(row, col, row*field_ratio, col*field_ratio, -1, n_background)
cm_images = np.zeros_like(pre_cm_images)

for i in range(n_background):
    cm_images[:,:,i] = contrast_modulate_lr(pre_cm_images[:,:,i], 4)

In [7]:
# spotters
uni_weighting_matrix = np.ones((row,col))
cm_weighting_matrix = luminance_modulate_lr(uni_weighting_matrix, 4, flip=True)
TM = Spotter(method='TM')
WTM = Spotter(method='WTM', whiten=1)
uni_RTM = Spotter(method='RTM', weight=uni_weighting_matrix)
cm_RTM = Spotter(method='RTM', weight=cm_weighting_matrix)
uni_WRTM = Spotter(method='WRTM', whiten=1, weight=uni_weighting_matrix)
cm_WRTM = Spotter(method='WRTM', weight=cm_weighting_matrix)

In [17]:
# simulation
def detection_1_f_noise(model, template, background, amp_init, n_iter):
    n_total = n_iter * n_background * 2
    amp = np.ones((n_total,)) * amp_init
    stimulus = np.ones((n_total,))
    response = np.ones((n_total,))
    amp_iter = np.ones((n_background * 2,))
    stimulus_iter = np.ones((n_background * 2,))
    response_iter = np.ones((n_background * 2,))

    for i in range(n_iter):
        for j in range(n_background):
            stimulus_iter[2*j] = 0
            response_iter[2*j] = model.give_response(background[:,:,j], template)
            stimulus_iter[2*j+1] = 1
            response_iter[2*j+1] = model.give_response(add_to_center(background[:,:,j], amp_iter[0]*template), template)
            
        amp[i*n_background*2:(i+1)*n_background*2] = amp_iter
        stimulus[i*n_background*2:(i+1)*n_background*2] = stimulus_iter
        response[i*n_background*2:(i+1)*n_background*2] = response_iter
        dp,_,_ = glm_cont_linear(amp_iter, stimulus_iter, response_iter)
        amp_iter[:] *= dp_tar / dp[0] * (np.random.rand() + 0.5)
    
    return amp, stimulus, response

In [20]:
# test
templates = [rc_target, sine_target, tri_target, sqr_target]
backgrounds = [uni_images, cm_images]
models = [TM, WTM, uni_RTM, uni_WRTM]
amp_init = 0.01

thresholds = np.zeros((len(templates), len(backgrounds), len(models)))
for t, template in enumerate(templates):
    for b, background in enumerate(backgrounds):
        thresholds[t,b,0] = linear_cont_th(*detection_1_f_noise(TM, template, background, amp_init, n_iter))
        thresholds[t,b,1] = linear_cont_th(*detection_1_f_noise(WTM, template, background, amp_init, n_iter))
        thresholds[t,b,2] = linear_cont_th(*detection_1_f_noise(uni_RTM, template, background, amp_init, n_iter)) if b==0 else linear_cont_th(*detection_1_f_noise(cm_RTM, template, background, amp_init, n_iter))
        thresholds[t,b,3] = linear_cont_th(*detection_1_f_noise(uni_WRTM, template, background, amp_init, n_iter)) if b==0 else linear_cont_th(*detection_1_f_noise(cm_WRTM, template, background, amp_init, n_iter))

TypeError: linear_cont_th() missing 2 required positional arguments: 'stimulus' and 'response'